In [1]:
import tensorflow as tf
import numpy as np
import sys

sys.path.append("..")
tf.reset_default_graph()

seed = 1234
dropout = 0.25
rnd = tf.set_random_seed(seed)

inputs = tf.placeholder(tf.float32, [None, 784], 'inputs')
targets = tf.placeholder(tf.float32, [None, 47], 'targets')

/afs/inf.ed.ac.uk/user/s17/s1758150/miniconda3/envs/mlp/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


# Affine Layer Example 

In [22]:
do1 = tf.layers.dropout(inputs, rate=dropout, seed=rnd, training=True)
w1 = tf.get_variable("weights1", shape=[784,100], initializer=tf.glorot_uniform_initializer())
b1 = tf.get_variable("bias1", shape=[100], initializer=tf.glorot_uniform_initializer())
relu1 = tf.nn.relu(tf.matmul(do1, w1)+b1)

do2 = tf.layers.dropout(relu1, rate=dropout, seed=rnd, training=True)
w2 = tf.get_variable("weights2", shape=[100,100], initializer=tf.glorot_uniform_initializer())
b2 = tf.get_variable("bias2", shape=[100], initializer=tf.glorot_uniform_initializer())
relu2 = tf.nn.relu(tf.matmul(do2, w2)+b2)

do3 = tf.layers.dropout(relu2, rate=dropout, seed=rnd, training=True)
w3 = tf.get_variable("weights3", shape=[100,47], initializer=tf.glorot_uniform_initializer())
b3 = tf.get_variable("bias3", shape=[47], initializer=tf.glorot_uniform_initializer())
outputs = tf.matmul(do3, w3)+b3

per_datapoint_errors = tf.nn.softmax_cross_entropy_with_logits(\
                        logits = outputs, labels =targets)
error = tf.reduce_mean(per_datapoint_errors)
per_datapoint_pred_is_correct = tf.equal(tf.argmax(outputs,1),\
                                         tf.argmax(targets,1))
accuracy = tf.reduce_mean(tf.cast(per_datapoint_pred_is_correct, tf.float32))

train_step = tf.train.GradientDescentOptimizer(learning_rate = 0.5).minimize(error)

sess = tf.InteractiveSession()

init_op = tf.global_variables_initializer()
sess.run(init_op)

import data_providers as data_providers

train_data = data_providers.EMNISTDataProvider('train', batch_size=50,\
                                              flatten=True, one_hot=True)
valid_data = data_providers.EMNISTDataProvider('valid', batch_size=50,\
                                              flatten=True, one_hot=True)

num_epoch = 10

for e in range(num_epoch):
    running_error=0
    for input_batch, target_batch in train_data:
        _, batch_error = sess.run([train_step, error], \
                                 feed_dict={inputs: input_batch, targets: target_batch})
        running_error += batch_error
    running_error /= train_data.num_batches
    print('End of epoch {0}: running error average = {1:.2f}'.format(e+1, running_error))

End of epoch 1: running error average = 1.77
End of epoch 2: running error average = 1.47
End of epoch 3: running error average = 1.43
End of epoch 4: running error average = 1.41
End of epoch 5: running error average = 1.38
End of epoch 6: running error average = 1.38
End of epoch 7: running error average = 1.37
End of epoch 8: running error average = 1.37
End of epoch 9: running error average = 1.36
End of epoch 10: running error average = 1.36


# CCN Structure from TF tutorial (but dataset is EMNIST)

In [4]:
input_layer = tf.reshape(inputs, [-1,28,28,1])
conv1 = tf.layers.conv2d(inputs = input_layer, filters=32,
                        kernel_size=[5,5], padding="same",
                        activation = tf.nn.relu)
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,2],
                               strides=2)
conv2 = tf.layers.conv2d(inputs = pool1, filters=64,
                        kernel_size=[5,5], padding="same",
                        activation = tf.nn.relu)
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2,2],
                               strides=2)

pool2_flat = tf.reshape(pool2, [-1,7*7*64])
dense = tf.layers.dense(inputs=pool2_flat, units=1024,
                        activation = tf.nn.relu)
dropout = tf.layers.dropout(inputs = dense, rate= 0.4, 
                           training=True)
outputs = tf.layers.dense(inputs=dropout, units=47)

per_datapoint_errors = tf.nn.softmax_cross_entropy_with_logits(\
                        logits = outputs, labels =targets)
error = tf.reduce_mean(per_datapoint_errors)
per_datapoint_pred_is_correct = tf.equal(tf.argmax(outputs,1),\
                                         tf.argmax(targets,1))
accuracy = tf.reduce_mean(tf.cast(per_datapoint_pred_is_correct, tf.float32))

train_step = tf.train.GradientDescentOptimizer(learning_rate = 0.5).minimize(error)

sess = tf.InteractiveSession()

init_op = tf.global_variables_initializer()
sess.run(init_op)

import data_providers as data_providers

train_data = data_providers.EMNISTDataProvider('train', batch_size=50,\
                                              flatten=True, one_hot=True)
valid_data = data_providers.EMNISTDataProvider('valid', batch_size=50,\
                                              flatten=True, one_hot=True)

num_epoch = 10

for e in range(num_epoch):
    running_error=0
    for input_batch, target_batch in train_data:
        _, batch_error = sess.run([train_step, error], \
                                 feed_dict={inputs: input_batch, targets: target_batch})
        running_error += batch_error
    running_error /= train_data.num_batches
    print('End of epoch {0}: running error average = {1:.2f}'.format(e+1, running_error))

End of epoch 1: running error average = 0.93
End of epoch 2: running error average = 0.50
End of epoch 3: running error average = 0.44
End of epoch 4: running error average = 0.40
End of epoch 5: running error average = 0.38
End of epoch 6: running error average = 0.36
End of epoch 7: running error average = 0.34
End of epoch 8: running error average = 0.33
End of epoch 9: running error average = 0.32
End of epoch 10: running error average = 0.31


# 1-layer CCN Example

In [9]:
input_layer = tf.reshape(inputs, [-1,28,28,1])
conv1 = tf.layers.conv2d(inputs = input_layer, filters=5,
                        kernel_size=[5,5], padding="valid",
                        activation = tf.nn.relu)
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,2],
                               strides=2)

pool1_flat = tf.reshape(pool1, [-1,5*12*12])
outputs = tf.layers.dense(inputs=pool1_flat, units=47)

per_datapoint_errors = tf.nn.softmax_cross_entropy_with_logits(\
                        logits = outputs, labels =targets)
error = tf.reduce_mean(per_datapoint_errors)
per_datapoint_pred_is_correct = tf.equal(tf.argmax(outputs,1),\
                                         tf.argmax(targets,1))
accuracy = tf.reduce_mean(tf.cast(per_datapoint_pred_is_correct, tf.float32))

train_step = tf.train.GradientDescentOptimizer(learning_rate = 0.5).minimize(error)

sess = tf.InteractiveSession()

init_op = tf.global_variables_initializer()
sess.run(init_op)

import data_providers as data_providers

train_data = data_providers.EMNISTDataProvider('train', batch_size=100,\
                                              flatten=True, one_hot=True)
valid_data = data_providers.EMNISTDataProvider('valid', batch_size=100,\
                                              flatten=True, one_hot=True)

num_epoch = 10

for e in range(num_epoch):
    running_error=0
    for input_batch, target_batch in train_data:
        _, batch_error = sess.run([train_step, error], \
                                 feed_dict={inputs: input_batch, targets: target_batch})
        running_error += batch_error
    running_error /= train_data.num_batches
    print('End of epoch {0}: running error average = {1:.2f}'.format(e+1, running_error))

End of epoch 1: running error average = 1.07
End of epoch 2: running error average = 0.73
End of epoch 3: running error average = 0.67
End of epoch 4: running error average = 1.05
End of epoch 5: running error average = 0.95
End of epoch 6: running error average = 0.89
End of epoch 7: running error average = 0.82
End of epoch 8: running error average = 0.69
End of epoch 9: running error average = 0.66
End of epoch 10: running error average = 0.64


# 2-layer CCN Example

In [11]:
input_layer = tf.reshape(inputs, [-1,28,28,1])
conv1 = tf.layers.conv2d(inputs = input_layer, filters=5,
                        kernel_size=[5,5], padding="valid",
                        activation = tf.nn.relu)
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,2],
                               strides=2)

conv2 = tf.layers.conv2d(inputs = pool1, filters=10,
                        kernel_size=[5,5], padding="valid",
                        activation = tf.nn.relu)
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2,2],
                               strides=2)

pool2_flat = tf.reshape(pool2, [-1,10*4*4])
outputs = tf.layers.dense(inputs=pool2_flat, units=47)

per_datapoint_errors = tf.nn.softmax_cross_entropy_with_logits(\
                        logits = outputs, labels =targets)
error = tf.reduce_mean(per_datapoint_errors)
per_datapoint_pred_is_correct = tf.equal(tf.argmax(outputs,1),\
                                         tf.argmax(targets,1))
accuracy = tf.reduce_mean(tf.cast(per_datapoint_pred_is_correct, tf.float32))

train_step = tf.train.GradientDescentOptimizer(learning_rate = 0.5).minimize(error)

sess = tf.InteractiveSession()

init_op = tf.global_variables_initializer()
sess.run(init_op)

import data_providers as data_providers

train_data = data_providers.EMNISTDataProvider('train', batch_size=100,\
                                              flatten=True, one_hot=True)
valid_data = data_providers.EMNISTDataProvider('valid', batch_size=100,\
                                              flatten=True, one_hot=True)

num_epoch = 10

for e in range(num_epoch):
    running_error=0
    for input_batch, target_batch in train_data:
        _, batch_error = sess.run([train_step, error], \
                                 feed_dict={inputs: input_batch, targets: target_batch})
        running_error += batch_error
    running_error /= train_data.num_batches
    print('End of epoch {0}: running error average = {1:.2f}'.format(e+1, running_error))

KeyboardInterrupt: 

In [10]:
def get_error_and_accuracy(data):
    err = 0
    acc = 0
    for input_batch, target_batch in data:
        e, a = sess.run([error, accuracy], feed_dict={inputs: input_batch, targets: target_batch})    
        err+=e
        acc+=a
    err /= data.num_batches
    acc /= data.num_batches
    
    return err, acc

print('Train data Error = {0:.2f} Accuracy={1:.2f}'.format(*get_error_and_accuracy(train_data)))
print('Valid data Error = {0:.2f} Accuracy={1:.2f}'.format(*get_error_and_accuracy(valid_data)))


Train data Error = 1.16 Accuracy=0.67
Valid data Error = 1.26 Accuracy=0.65


In [ ]:
default_graph = tf.get_default_graph()
print('Number of operations in graph: {0}'.format(len(default_graph.get_operations())))
graph = tf.Graph()